## Adding dates to the training data

In [1]:
import json
import os

from collections import defaultdict

In [2]:
path = "yearly_data"
f_list = sorted(os.listdir(path))

mapping = {
    "1B":"2012-12",
    "2B":"2013-12",
    "3B":"2014-12",
    "4B":"2015-12",
    "5B":"2016-12",
    "6B":"2017-12",
    "7B":"2018-12",
    "8B":"2019-12"
}

In [3]:
with open("training9b.json") as f:
    train9b = json.load(f)["questions"]
    
train9b_ids = {x["id"] for x in train9b}

In [4]:
len(train9b_ids)

3743

In [5]:
all_test_queries = {}

for f_name in f_list:
    with open(os.path.join(path, f_name)) as f:
        for query_data in json.load(f)["questions"]:
            _temp = {}
            _temp["phase"] = f_name[:2]
            _temp["limit_date"] = mapping[_temp["phase"]]
            all_test_queries[query_data["id"]] = _temp
            
len(all_test_queries)

3800

In [6]:
for q_data in train9b:
    try:
        q_data["phase"] = all_test_queries[q_data["id"]]["phase"]
        q_data["limit_date"] = all_test_queries[q_data["id"]]["limit_date"]
    except:
        print(q_data["body"])
  
len(train9b)

3743

In [7]:
with open("training9b_wDates.json","w") as f:
    json.dump({"questions":train9b},f)

## Split the data

In [12]:
test_8b = []

for i in range(1,6):
    id_sets = set()
    with open(f"yearly_data/8B{i}_golden.json","r") as f:
        for q in json.load(f)["questions"]:
            id_sets.add(q["id"])
            
    test_8b.append(id_sets)

print(sum([len(x) for x in test_8b]))

500


## Statistics

In [17]:
from utils import collection_iterator, f_map, create_filter_query_function, to_date

articles = sum(collection_iterator("/backup/pubmed_archive_json/bioasq_9b.tar.gz"), [])
articles = {x["id"]:x for x in articles}

[CORPORA] Openning tar file /backup/pubmed_archive_json/bioasq_9b.tar.gz
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_00000000_to_03000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_03000000_to_06000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_06000000_to_09000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_09000000_to_12000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_12000000_to_15000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_15000000_to_18000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_18000000_to_21000000
[CORPORA] Openning tar file tmp/tmp5bgbvft3/bioasq_9b_21000000_to_21165909


In [31]:
documents_not_present = 0
valid_document = 0
invalid_document = 0

for q_data in train9b:
    for d in q_data["documents"]:
        _id = d.split("/")[-1]
        if _id in articles:
            if to_date(q_data["limit_date"]) >= to_date(articles[_id]["pubdate"]):
                valid_document += 1
            else:
                invalid_document += 1
        else:
            documents_not_present+=1

print("documents_not_present",documents_not_present)
print("invalid_document", invalid_document)
print("valid_document", valid_document)

documents_not_present 108
invalid_document 1038
valid_document 34139


datetime.datetime(2014, 12, 1, 0, 0)

False